# Chapter 4: Computing sea ice extent and time series graph
Creators: Ann Windnagel and Robyn Marowitz

Affiliation: [National Snow And Ice Data Center](https://nsidc.org/home)

To download this data a user will need an EARTHDATA Login. https://nsidc.org/data/nsidc-0771/versions/1#anchor-data-access-tools is the link to access. For the ease of this notebook we have downloaded what is needed for this specific work and placed it in the `ancillary_files` folder in this repo.

In [1]:
import glob
import os
import pystac
import fsspec
import os.path as op
import csv
import numpy as np
import xarray as xr
import numpy.ma as ma
from netCDF4 import Dataset
from datetime import datetime, timedelta, date
import pandas as pd
import matplotlib.pyplot as plt
import re

To start we will once again download our desired file like we did in the previous notebooks.

In [2]:
collection = pystac.Collection.from_file('https://noaadata.apps.nsidc.org/NOAA/G02202_V4/stac/collection.json')
daily_nh_2023_item = collection.get_item("seaice_conc_daily_nh_2023_v04r00", recursive=True)
netcdf_url = daily_nh_2023_item.assets['netcdf'].href
fs = fsspec.filesystem('https')
ds = xr.open_dataset(fs.open(netcdf_url))
ds

<xarray.Dataset> Size: 2GB
Dimensions:                         (tdim: 365, y: 448, x: 304)
Coordinates:
    time                            (tdim) datetime64[ns] 3kB ...
    xgrid                           (x) float32 1kB ...
    ygrid                           (y) float32 2kB ...
Dimensions without coordinates: tdim, y, x
Data variables:
    cdr_seaice_conc                 (tdim, y, x) float32 199MB ...
    melt_onset_day_cdr_seaice_conc  (tdim, y, x) float32 199MB ...
    nsidc_bt_seaice_conc            (tdim, y, x) float32 199MB ...
    nsidc_nt_seaice_conc            (tdim, y, x) float32 199MB ...
    projection                      |S1 1B ...
    qa_of_cdr_seaice_conc           (tdim, y, x) float32 199MB ...
    spatial_interpolation_flag      (tdim, y, x) float32 199MB ...
    stdev_of_cdr_seaice_conc        (tdim, y, x) float32 199MB ...
    temporal_interpolation_flag     (tdim, y, x) float32 199MB ...
    latitude                        (y, x) float32 545kB ...
    longitude                       (y, x) float32 545kB ...
Attributes: (12/39)
    Conventions:               CF-1.6, ACDD-1.3
    title:                     NOAA/NSIDC Climate Data Record of Passive Micr...
    references:                Comiso, J. C., and F. Nishio. 2008. Trends in ...
    comment:                   none
    program:                   NOAA Climate Data Record Program
    cdr_variable:              cdr_seaice_conc
    ...                        ...
    acknowledgment:            This project was supported in part by a grant ...
    platform:                  DMSP 5D-3/F17 > Defense Meteorological Satelli...
    sensor:                    SSMI/S > Special Sensor Microwave Imager/Sounder
    date_created:              2023-09-26T20:14:01Z
    time_coverage_start:       2023-01-01T00:00:00Z
    time_coverage_end:         2023-12-31T23:59:59Z

In [3]:
# Pull out sea ice concentration variable into numpy array
cdr_seaice_conc_var = ds.variables['cdr_seaice_conc']
type(cdr_seaice_conc_var)

xarray.core.variable.Variable

In [4]:
cdr_seaice_conc = np.array(cdr_seaice_conc_var)
type(cdr_seaice_conc)

numpy.ndarray

## Get northern ancillary file and save it as a NetCDF4 Dataset

In [5]:
pixel_area_fn = 'ancillary_files/NSIDC0771_CellArea_PS_N25km_v1.0.nc'
pixel_area_nc = Dataset(pixel_area_fn, 'r')
pixel_area_nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    title: Polar Stereo Ancillary Grid Information
    summary: This data set provides the total on-Earth surface area and the the latitude and longitude values at the center of each grid cell of the 6.25km, 12.5km, and 25km polar stereographic gridded data sets distributed by The National Snow and Ice Data Center
    id: 10.5067/N6INPBT8Y104
    metadata_link: https://doi.org/10.5067/N6INPBT8Y104
    license: Access Constraint: These data are freely, openly, and fully accessible, provided that you are logged into your NASA Earthdata profile (https://urs.earthdata.nasa.gov/);  Use Constraint: These data are freely, openly, and fully available to use without restrictions, provided that you cite the data according to the recommended citation at https://nsidc.org/about/use_copyright.html. For more information on the NASA EOSDIS Data Use Policy, see https://earthdata.nasa.gov/earth-observation-data/data-u

In [6]:
# Get cell area and place in an array
pixel_area = pixel_area_nc.variables['cell_area']

# We divide by 1 million to convert from square meters to square kilometers
pixel_area_scaled = np.array(pixel_area)/1000000
pixel_area_scaled

array([[382.65885395, 383.38481392, 384.10806602, ..., 386.97334454,
        386.26124949, 385.54631636],
       [383.76521935, 384.49431511, 385.22069419, ..., 388.09838899,
        387.38320398, 386.66517135],
       [384.87158575, 385.60382172, 386.3333322 , ..., 389.223461  ,
        388.50518158, 387.78404504],
       ...,
       [406.944431  , 407.74022873, 408.53312612, ..., 411.67498178,
        410.89405102, 410.1100738 ],
       [405.84558089, 406.63817365, 407.42787474, ..., 410.55703547,
        409.77926452, 408.99845652],
       [404.74599928, 405.53538917, 406.32189606, ..., 409.43837039,
        408.66375711, 407.88611619]])

In [7]:
daily_nh_2023_item.properties['end_datetime']

'2023-12-31T23:59:59Z'

In [8]:
daily_nh_2023_item.properties['start_datetime']

'2023-01-01T00:00:00Z'

In [9]:
cdr_seaice_conc.shape

(365, 448, 304)

In [10]:
len(cdr_seaice_conc)

365

In [11]:
file_time = ds.variables['time']
file_time

<xarray.Variable (tdim: 365)> Size: 3kB
[365 values with dtype=datetime64[ns]]
Attributes:
    standard_name:  time
    long_name:      ANSI date
    axis:           T
    cell_methods:   tdim: mean

In [22]:
# Get year from STAC item property
year = daily_nh_2023_item.properties['start_datetime'][:4]

In [23]:
d = []
# Get daily extent and save dataframe
for t in range(len(cdr_seaice_conc)):
    # Here we are squeezing to be 2d for each day
    cdr_seaice_conc_2d = np.copy(cdr_seaice_conc[t,:,:].squeeze())

    # Compute Extent
    # Create mask to filter out the flag values from the computation and concentration equal to or less than 15%
    # Flag values are 2.52, 2.53, 2.54, 2.55; Note: not removing 2.51 because that's the pole hole and it is
    # included in the extent total because the ice there is assumed to have a concentration > 15%
    # Note that not removing 251 likely doesn't do anything because there are no 2.51's in v04 since we
    # spatially interpolate the pole hole now.
    # This line removes flag values and concentration < 15%
    flag_mask_extent = ma.masked_outside(cdr_seaice_conc_2d, 0.15, 2.51)

    non_mask_extent = ma.nonzero(flag_mask_extent)
    flag_mask_extent[non_mask_extent] = 1.0 
    sea_ice_extent_raster = flag_mask_extent * pixel_area_scaled
    cdr_seaice_exent_total = sea_ice_extent_raster.sum()
    doy = t + 1 
    date = datetime.datetime.strptime(f'{year} {doy}', '%Y %j').strftime('%Y/%m/%d')
    # print(date, ':', cdr_seaice_exent_total)
    d.append(
        {
            'Date (yyyy/mm/dd)': date, 
            'Extent (km2)': cdr_seaice_exent_total
        }
    )

# Create data frame from list
extent_df = pd.DataFrame(d)
extent_df
    

,Date (yyyy/mm/dd),Extent (km2)
0,2023/01/01,1.270498e+07
1,2023/01/02,1.281669e+07
2,2023/01/03,1.286228e+07
3,2023/01/04,1.289838e+07
4,2023/01/05,1.298824e+07
...,...,...
360,2023/12/27,1.307201e+07
361,2023/12/28,1.304457e+07
362,2023/12/29,1.310637e+07
363,2023/12/30,1.312580e+07


## NOTE we are already scaled in the netcdf and are leaving it instead of multiplying by 100
leaving it as a decimal instead of a percent

In [24]:
cdr_seaice_conc_2d = np.copy(cdr_seaice_conc[1,:,:].squeeze())
cdr_seaice_conc_2d.max()

np.float32(2.54)